In [1]:
from itertools import product, combinations
from collections import deque, defaultdict

from context import samsurf
from triangulation import Triangle, Triangulation
import geom_equiv
from polygon import Point
from halfplane import HalfPlane

ModuleNotFoundError: No module named 'bowman'

# Messing with the Spine

In [62]:
def find_trin_good(X):
    r"""
    Find a Delaunay triangulation of ``X`` that permits standard ``gt``-flow for time epsilon.

    .. NOTE:

        This means flowing downward from i in the hyperbolic plane.
    """
    perturb = matrix(AA, [[1, 0], [0, 1 - 1/10_000]])
    if X.apply_matrix(perturb).is_delaunay:
        return X
    r = X.idr
    if any(edge.end == Point(0, 1) for edge in r.polygon.edges):
        idx_edge_ingoing = next(k for k, edge in enumerate(r.polygon.edges) if edge.end == Point(0, 1))
        return find_trin_good(r.get_trin_neighboring(idx_edge_ingoing))
    else:
        # i is interior to an edge of a bad IDR
        rotate = matrix(AA, [[0, -1], [1, 0]])
        return X.apply_matrix(rotate)

In [63]:
def directions_shortest(X):
    dirns = {}
    for (idx_t, idx_e) in X.idxs_systoles:
        edge = X.triangles[idx_t][idx_e]
        dirn = oo if edge[0] == 0 else edge[1]/edge[0]
        dirns[dirn] = (idx_t, idx_e)
    return dirns

In [64]:
def systoles_steepest(X):
    steepness_to_systoles = defaultdict(list)
    for idxs in X.idxs_systoles:
        edge = X.triangles[idxs[0]][idxs[1]]
        steepness = oo if edge[0] == 0 else abs(edge[1]/edge[0])
        steepness_to_systoles[steepness].append(idxs)
    steepest = max(steepness_to_systoles)
    return sorted(steepness_to_systoles[steepest])

In [65]:
def exit_factor(r):
    axis_imag = HalfPlane.from_ineq(0, 1, 0)
    points_intersection = [edge.halfplane.intersect_boundaries(axis_imag) for edge in r.polygon]
    m = min(pt.v2 for pt in points_intersection if pt != oo and pt is not None)
    return sqrt(AA(m))

In [66]:
def traverse_edge(X, k=1):
    X = find_trin_good(X)
    idxs_systole = systoles_steepest(X)[0]
    next_ups = X.idxs_tying(idxs_systole)
    if not next_ups:
        ef = exit_factor(X.idr)
        X = X.apply_matrix(matrix([[1, 0], [0, ef]]))
        return traverse_edge(X, k * ef)
    else:
        idxs_next_up = next_ups[0]
        k1 = X.contraction_factor(idxs_systole, idxs_next_up)
        m_trans = matrix(AA, [[1, 0], [0, k1]])
        X = X.apply_matrix(m_trans)
        assert len(directions_shortest(X)) > 2
        return k * k1

In [67]:
def get_neighbors(X0, m0):
    X = X0.apply_matrix(m0).make_delaunay()
    assert len(directions_shortest(X)) > 2
    neighbors = []
    for idxs0, idxs1 in combinations(sorted(directions_shortest(X).values()), r=2):
        edge0, edge1 = X.triangles[idxs0[0]][idxs0[1]], X.triangles[idxs1[0]][idxs1[1]]
        if matrix([edge0, edge1]).determinant() < 0:
            edge0, edge1 = edge1, edge0
        if edge0.dot_product(edge1) > 0:
            edge0, edge1 = -edge1, edge0

        J = matrix(AA, [[0, -1], [1, 0]])
        m_align = matrix(AA, [edge0 + edge1, J*(edge0 + edge1)]).T**(-1)
        X_aligned = X.apply_matrix(m_align)

        if [idxs0, idxs1] == systoles_steepest(X_aligned):
            k = traverse_edge(X_aligned)
            m_total = m_align**(-1) * matrix(AA, [[1, 0], [0, k]]) * m_align * m0
            neighbors.append(m_total)
        elif [idxs0, idxs1] == systoles_steepest(X_aligned.apply_matrix(J)):
            k = traverse_edge(X_aligned.apply_matrix(J))
            m_total = m_align**(-1) * J**(-1) * matrix(AA, [[1, 0], [0, k]]) * J * m_align * m0
            neighbors.append(m_total)
        else:
            continue

    return neighbors

In [68]:
def explore_spine(X0, num_visited=10):
    visited = set()
    to_visit = deque([identity_matrix(AA, 2)])
    k = 0
    while k < num_visited:
        m = to_visit.pop()
        m.set_immutable()
        visited.add(m)
        for m1 in get_neighbors(X0, m):
            m1.set_immutable()
            if m1 not in visited:
                to_visit.appendleft(m1)
        k += 1
    return visited

In [69]:
def double_pentagon():
    X = Triangulation.ronen_l(5)
    tris_new = []
    for tri in X.triangles:
        edges_new = [vector(AA, [x, y]) for x, y in tri]
        tris_new.append(Triangle(*edges_new))
    Xint = Triangulation(tris_new, X.gluings)

    [idx_systole] = Xint.idxs_systoles
    [idx_tying] = Xint.idxs_tying(idx_systole)
    k = Xint.contraction_factor(idx_systole, idx_tying)
    Xedge = Xint.apply_matrix(matrix([[1, 0], [0, k]]))

    edge_h, edge_v = Xedge.triangles[2][2], Xedge.triangles[4][0]
    a, b = edge_h + edge_v
    m_norm = matrix([[a, -b], [b, a]])**(-1)
    Xedge = find_trin_good(Xedge.apply_matrix(m_norm))

    idx_sys = Xedge.idxs_systoles[0]
    idx_tying = Xedge.idxs_tying(idx_sys)[0]
    k = Xedge.contraction_factor(idx_sys, idx_tying)
    m_trans = matrix([[1, 0], [0, k]])

    Xvert = Xedge.apply_matrix(m_norm**-1 * m_trans).make_delaunay()
    # tilting one side horizontal
    v, w = Xvert.triangles[5][1], Xvert.triangles[1][1]
    J = matrix(AA, [[0, -1], [1, 0]])
    m = matrix(AA, [-J*(v + w), v + w]).T.inverse()
    return Xvert.apply_matrix(m)

In [70]:
def main():
    X = double_pentagon()
    print("starting")
    verts = explore_spine(X, 6)

%prun -D temp.dat main()

starting
 
*** Profile stats marshalled to file 'temp.dat'. 


         15682373 function calls (15637671 primitive calls) in 26.831 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   206484    3.964    0.000    9.478    0.000 qqbar.py:7806(_interval_fast)
  1406874    1.997    0.000    1.997    0.000 sets_cat.py:972(_element_constructor_from_element_class)
   217639    1.536    0.000    3.117    0.000 {method 'polynomial' of 'sage.rings.number_field.number_field_element.NumberFieldElement' objects}
   170782    1.353    0.000    1.619    0.000 number_field.py:1829(_convert_non_number_field_element)
235668/232747    1.198    0.000    1.485    0.000 polynomial_ring.py:309(_element_constructor_)
     3549    1.115    0.000    1.875    0.001 qqbar.py:7049(_real_refine_interval)
      732    0.978    0.001    1.399    0.002 number_field.py:10333(_factor_univariate_polynomial)
   322609    0.954    0.000   11.037    0.000 qqbar.py:3590(__init__)
   222312    0.938    0.000    5.665    0.000